In [2]:
import pandas as pd

In [90]:
election2016 = pd.read_excel('federalelections2016.xlsx', header=3, nrows=51,
                            sheet_name='Table 2. Electoral &  Pop Vote')

# Crete column index
election2016.set_axis(["State", "Electoral Republican", "Electorial Democrats", 
                   "Popular Republican", "Popular Democrats", "Popular Other", "Popular Total"], 
                      axis=1, inplace=True)

# Figure R vs D state
election2016['Party'] = election2016.apply(lambda x:"R" if x['Popular Republican'] > x['Popular Democrats'] else "D", 
                                           axis=1)
# Only keep party information
election2016=election2016[['State', 'Party']]

# Reset index
election2016.set_index('State', inplace=True)

In [92]:
election2016.head(10)

,Party
State,
AL,R
AK,R
AZ,R
AR,R
CA,D
CO,D
CT,D
DE,D
DC,D


In [93]:
covid = pd.read_csv('United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv')

# Only care about number of new death
covid = covid[['submission_date', 'state', 'new_death']]

# Rename columns
covid.set_axis(['Submission Date', 'State', 'Death'], axis=1, inplace=True)

# Reindex
covid = covid.set_index(['State', 'Submission Date'])

In [96]:
covid.head(10)

Death
State Submission Date       
CO    01/22/2020           0
      01/23/2020           0
      01/24/2020           0
      01/25/2020           0
      01/26/2020           0
      01/27/2020           0
      01/28/2020           0
      01/29/2020           0
      01/30/2020           0
      01/31/2020           0